In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# работа с базой данных и pandas

In [ ]:
conn = psycopg2.connect(
    host="158.160.52.106",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)
cur = conn.cursor()

In [ ]:
query_w = '''SELECT DATE_TRUNC('week', visit_dttm) as weeks,
        COUNT(distinct Cl.client_rk) as visitors,
        COUNT(distinct Ac.client_rk) as accounts,
        COUNT(distinct Ap.account_rk) as applications,
        COUNT(distinct Ap.account_rk) filter(where game_flg = 1) as happened_games
    FROM msu_analytics.Client as Cl
    LEFT JOIN msu_analytics.Account as Ac using(client_rk)
    LEFT JOIN msu_analytics.Application as Ap using(account_rk)
    LEFT JOIN msu_analytics.Game using(game_rk)
    GROUP BY weeks ORDER BY weeks'''

In [ ]:
# dataframe по неделям

dfw = pd.read_sql(query_w, conn)
dfw.head()

,weeks,visitors,accounts,applications,happened_games
0,2022-08-29,111,32,7,5
1,2022-09-05,184,31,6,5
2,2022-09-12,167,43,16,8
3,2022-09-19,181,45,12,11
4,2022-09-26,195,59,17,17


In [ ]:
# dataframe конверсии по неделям

cw = pd.DataFrame()
cw['weeks'] = dfw.weeks
cw['visitors'] = 100
cw['accounts'] = round(dfw.accounts/dfw.visitors * 100, 2)
cw['applications'] = round(dfw.applications/dfw.visitors * 100, 2)
cw['happened_games'] = round(dfw.happened_games/dfw.visitors * 100, 2)
cw.head()

,weeks,visitors,accounts,applications,happened_games
0,2022-08-29,100,28.83,6.31,4.50
1,2022-09-05,100,16.85,3.26,2.72
2,2022-09-12,100,25.75,9.58,4.79
3,2022-09-19,100,24.86,6.63,6.08
4,2022-09-26,100,30.26,8.72,8.72


In [ ]:
query_m = '''SELECT DATE_TRUNC('month', visit_dttm) as months,
        COUNT(distinct Cl.client_rk) as visitors,
        COUNT(distinct Ac.client_rk) as accounts,
        COUNT(distinct Ap.account_rk) as applications,
        COUNT(distinct Ap.account_rk) filter(where game_flg = 1) as happened_games
    FROM msu_analytics.Client as Cl
    LEFT JOIN msu_analytics.Account as Ac using(client_rk)
    LEFT JOIN msu_analytics.Application as Ap using(account_rk)
    LEFT JOIN msu_analytics.Game using(game_rk)
    GROUP BY months ORDER BY months'''

In [ ]:
# dataframe по месяцам

dfm = pd.read_sql(query_m, conn)
dfm.months = ['September', 'October', 'November', 'December', 'January', 'February']
dfm

,months,visitors,accounts,applications,happened_games
0,September,778,190,51,39
1,October,843,217,65,56
2,November,786,200,56,50
3,December,805,196,57,33
4,January,852,213,64,20
5,February,32,8,0,0


In [ ]:
# dataframe конверсии по месяцам

cm = pd.DataFrame()
cm['months'] = dfm.months
cm['visitors'] = 100
cm['accounts'] = round(dfm.accounts/dfm.visitors * 100, 2)
cm['applications'] = round(dfm.applications/dfm.visitors * 100, 2)
cm['happened_games'] = round(dfm.happened_games/dfm.visitors * 100, 2)
cm

,months,visitors,accounts,applications,happened_games
0,September,100,24.42,6.56,5.01
1,October,100,25.74,7.71,6.64
2,November,100,25.45,7.12,6.36
3,December,100,24.35,7.08,4.10
4,January,100,25.00,7.51,2.35
5,February,100,25.00,0.00,0.00


# визуализация

In [ ]:
x = list(dfm.iloc[:, 0])

fig = go.Figure(go.Bar(x=x, y=dfm.iloc[:, 1], name=dfm.columns[1], legendgroup="group1", marker=dict(color='cornflowerblue')))
fig.add_trace(go.Bar(x=x, y=dfm.iloc[:, 2], name=dfm.columns[2], legendgroup="group1", marker=dict(color='indianred')))
fig.add_trace(go.Bar(x=x, y=dfm.iloc[:, 3], name=dfm.columns[3], legendgroup="group1", marker=dict(color='darkseagreen')))
fig.add_trace(go.Bar(x=x, y=dfm.iloc[:, 4], name=dfm.columns[4], legendgroup="group1", marker=dict(color='orchid')))

fig.add_traces([go.Scatter(x=x, y=cm.iloc[:, 2], yaxis="y2", name="Accounts", marker=dict(color="crimson"), legendgroup="group2"),
                go.Scatter(x=x, y=cm.iloc[:, 3], yaxis="y2", name="Applications", marker=dict(color="darkgreen"), legendgroup="group2"),
                go.Scatter(x=x, y=cm.iloc[:, 4], yaxis="y2", name="Happened games", marker=dict(color="darkmagenta"), legendgroup="group2")])

fig.update_layout(
    title=dict(text="Customer journey (by months)",
               font=dict(size=30),
               x = 0.49,
               yref='paper'),
    legend=dict(orientation="h",
                entrywidth=160,
                xanchor="left",
                x=0.34),
    yaxis=dict(title=dict(text="Amount"),
               side="left",
               range=[0, 920]),
    yaxis2=dict(title=dict(text="Conversion, %"),
                side="right",
                range=[0, 92],
                overlaying="y",
                tickmode="sync"),
    barmode='group')

fig.show()

In [ ]:
x = list(dfw.iloc[:, 0])

fig = go.Figure(go.Bar(x=x, y=dfw.iloc[:, 1], name=dfw.columns[1], legendgroup="group1", marker=dict(color='cornflowerblue')))
fig.add_trace(go.Bar(x=x, y=dfw.iloc[:, 2], name=dfw.columns[2], legendgroup="group1", marker=dict(color='indianred')))
fig.add_trace(go.Bar(x=x, y=dfw.iloc[:, 3], name=dfw.columns[3], legendgroup="group1", marker=dict(color='darkseagreen')))
fig.add_trace(go.Bar(x=x, y=dfw.iloc[:, 4], name=dfw.columns[4], legendgroup="group1", marker=dict(color='orchid')))

fig.add_traces([go.Scatter(x=x, y=cw.iloc[:, 2], yaxis="y2", name="Accounts", marker=dict(color="crimson"), legendgroup="group2"),
                go.Scatter(x=x, y=cw.iloc[:, 3], yaxis="y2", name="Applications", marker=dict(color="darkgreen"), legendgroup="group2"),
                go.Scatter(x=x, y=cw.iloc[:, 4], yaxis="y2", name="Happened games", marker=dict(color="darkmagenta"), legendgroup="group2")])

fig.update_layout(
    title=dict(text="Customer journey (by weeks)",
               font=dict(size=30),
               x = 0.49,
               yref='paper'),
    legend=dict(orientation="h",
                entrywidth=160,
                xanchor="left",
                x=0.34),
    yaxis=dict(title=dict(text="Amount"),
               side="left",
               range=[0, 220]),
    yaxis2=dict(title=dict(text="Conversion, %"),
                side="right",
                range=[0, 44],
                overlaying="y",
                tickmode="sync"),
    barmode='group')

fig.show()

>**Еще одна версия визуализации**

In [ ]:
x = list(dfm.iloc[:, 0])

fig = go.Figure(go.Scatter(x=x, y=dfm['visitors'], text= 'Conversion: 100%',
                           fill='tozeroy', mode='lines', name='Visitors'))

fig.add_traces([go.Scatter(x=x, y=dfm['accounts'],
                    text= 'Conversion: ' + cm['accounts'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Accounts'),
                go.Scatter(x=x, y=dfm['applications'],
                    text= 'conversion: ' + cm['applications'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Applications'),
                go.Scatter(x=x, y=dfm['happened_games'],
                    text= 'conversion: ' + cm['happened_games'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Happened games')])

fig.update_yaxes(title_text='Amount')
fig.update_layout(title=dict(text="Customer journey (by months)", font=dict(size=30), x = 0.49, yref='paper'))

fig.show()

In [ ]:
x = list(dfw.iloc[:, 0])

fig = go.Figure(go.Scatter(x=x, y=dfw['visitors'], text= 'Conversion: 100%',
                           fill='tozeroy', mode='lines', name='Visitors'))

fig.add_traces([go.Scatter(x=x, y=dfw['accounts'],
                    text= 'Conversion: ' + cw['accounts'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Accounts'),
                go.Scatter(x=x, y=dfw['applications'],
                    text= 'conversion: ' + cw['applications'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Applications'),
                go.Scatter(x=x, y=dfw['happened_games'],
                    text= 'conversion: ' + cw['happened_games'].astype(str) + '%',
                    fill='tozeroy', mode='lines', name='Happened games')])

fig.update_yaxes(title_text='Amount')
fig.update_layout(title=dict(text="Customer journey (by weeks)", font=dict(size=30), x = 0.49, yref='paper'))

fig.show()

# выводы

* Из графиков видно, что конверсия в регистрацию колеблется на уровне 25% – это отличный показатель, если сравнивать с аналогичными продуктами на рынке услуг. \
* Примерно каждый третий зарегистрировавшийся подает заявку на игру. Стоит проанализировать этот этап: может быть, клиент сталкивается со сложностями при попытке оставить заявку. Тогда следует упростить этот процесс, добавить альтернативные способы. \
* Лишь две трети запланированных игр в итоге случаются. Необходимо понять, почему клиент отменяет игру/не приходит. Можно внедрить смс-напоминания/письма о запланированной игре, звонки. При отмене просить указать причину, по которой клиент передумал. \

\

**Отдельно стоит отметить следущее:**
1. Во второй половине декабря – январе количество оправданных заявок сильно проседает по сравнению с осенними показателями. Важно выяснить, с чем это связано: с сезонностью или с изменением в работе с клиентами.
2. На стыке сентября и октября случился всплеск конверсии в регистрацию. Следует проверить, случайность это или чья-то заслуга.